<a href="https://colab.research.google.com/github/amaj8/Downton-Chatbot/blob/master/Downton_Chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F
from datetime import datetime as dt
import datetime
import re
from collections import Counter
import random
import itertools
import os

cuda = torch.cuda.is_available()
device = torch.device("cuda" if cuda else "cpu")

* Preprocess the data
* Model
* Loss function

In [2]:
f = open('downton.abbey.s01e01.srt','r')
content = f.read()
group = [group.split("\n",2) for group in content.split("\n\n")]
print(group)

[['1', '00:01:10,780 --> 00:01:13,060', 'Oh, my God!'], ['2', '00:01:16,580 --> 00:01:18,580', "That's impossible."], ['3', '00:01:19,280 --> 00:01:20,580', "I'll take it up there now."], ['4', '00:01:21,080 --> 00:01:23,690', "Don't be stupid. None of\nthem will be up for hours,"], ['5', '00:01:23,700 --> 00:01:25,340', 'and what difference will it make?'], ['6', '00:01:25,350 --> 00:01:27,360', "Jimmy'll do it when he comes in."], ['7', '00:01:41,470 --> 00:01:44,060', 'April 1912'], ['8', '00:01:45,420 --> 00:01:47,380', "Six o'clock!"], ['9', '00:01:49,220 --> 00:01:52,240', 'Thank you, Daisy... Anna?'], ['10', '00:01:56,120 --> 00:01:57,610', 'Just for once in my life,'], ['11', '00:01:57,620 --> 00:02:00,480', "I'd like to sleep\nuntil I woke up natural."], ['12', '00:02:01,380 --> 00:02:04,480', '- Is your fire still in?\n- Yes, Mrs. Patmore.'], ['13', '00:02:04,490 --> 00:02:06,630', 'Ooh, my, will wonders never cease?'], ['14', '00:02:06,660 --> 00:02:09,410', "- Have you laid

In [3]:
def dialogue_pairs(group):
  pairs = []
  for i in range(1,len(group)):
    if dt.strptime(group[i][1].split(" --> ")[0],'%H:%M:%S,%f') <= dt.strptime(group[i-1][1].split(" --> ")[1] \
                        ,'%H:%M:%S,%f') + datetime.timedelta(seconds=4):
      pairs.append([group[i-1][2],group[i][2]])
  return pairs

In [4]:
pairs1 = dialogue_pairs(group)

In [5]:
def clean_text(text):
    
    text = text.lower()
    
    text = re.sub(r"i'm", "i am", text)
    text = re.sub(r"he's", "he is", text)
    text = re.sub(r"she's", "she is", text)
    text = re.sub(r"it's", "it is", text)
    text = re.sub(r"that's", "that is", text)
    text = re.sub(r"what's", "that is", text)
    text = re.sub(r"where's", "where is", text)
    text = re.sub(r"how's", "how is", text)
    text = re.sub(r"\'ll", " will", text)
    text = re.sub(r"\'ve", " have", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"\'d", " would", text)
    text = re.sub(r"\'re", " are", text)
    text = re.sub(r"won't", "will not", text)
    text = re.sub(r"can't", "cannot", text)
    text = re.sub(r"n't", " not", text)
    text = re.sub(r"n'", "ng", text)
    text = re.sub(r"'bout", "about", text)
    text = re.sub(r"'til", "until", text)
    text = re.sub(r"[-()\"#/@;:<>{}`+=~|.!?,]", "", text)
    text = text.replace("\n"," ") 
    
    return text


In [6]:
for p in pairs1:
  p[0] = clean_text(p[0])
  p[1] = clean_text(p[1])

In [7]:
print(pairs1)

[['oh my god', 'that is impossible'], ['that is impossible', 'i will take it up there now'], ['i will take it up there now', 'do not be stupid none of them will be up for hours'], ['do not be stupid none of them will be up for hours', 'and what difference will it make'], ['and what difference will it make', 'jimmy will do it when he comes in'], ['april 1912', "six o'clock"], ["six o'clock", 'thank you daisy anna'], ['thank you daisy anna', 'just for once in my life'], ['just for once in my life', 'i would like to sleep until i woke up natural'], ['i would like to sleep until i woke up natural', ' is your fire still in  yes mrs patmore'], [' is your fire still in  yes mrs patmore', 'ooh my will wonders never cease'], ['ooh my will wonders never cease', " have you laid the servants' hall breakfast  yes mrs patmore"], [" have you laid the servants' hall breakfast  yes mrs patmore", ' and finished blacking that stove  yes mrs patmore'], [' and finished blacking that stove  yes mrs patmore'

In [8]:
THRESHOLD = 3
MAX_SENTENCE_LENGTH = 10
MAX_LENGTH = 10

In [9]:
pairs1 = [p for p in pairs1 if len(p[0].split()) <= MAX_SENTENCE_LENGTH and len(p[1].split()) <= MAX_SENTENCE_LENGTH]

In [10]:
pairs1

[['oh my god', 'that is impossible'],
 ['that is impossible', 'i will take it up there now'],
 ['and what difference will it make', 'jimmy will do it when he comes in'],
 ['april 1912', "six o'clock"],
 ["six o'clock", 'thank you daisy anna'],
 ['thank you daisy anna', 'just for once in my life'],
 ['just for once in my life', 'i would like to sleep until i woke up natural'],
 ['i would like to sleep until i woke up natural',
  ' is your fire still in  yes mrs patmore'],
 [' is your fire still in  yes mrs patmore',
  'ooh my will wonders never cease'],
 ['ooh my will wonders never cease',
  " have you laid the servants' hall breakfast  yes mrs patmore"],
 [" have you laid the servants' hall breakfast  yes mrs patmore",
  ' and finished blacking that stove  yes mrs patmore'],
 [' and finished blacking that stove  yes mrs patmore',
  ' what about the bedroom fires  all lit mrs patmore'],
 [' what about the bedroom fires  all lit mrs patmore',
  'right well take the things'],
 ['right wel

In [11]:
queries = [p[0] for p in pairs1]
responses = [p[1] for p in pairs1]
print(queries)
query_words = []
# query_words = [q.split() for q in queries]
for q in queries:
  query_words.extend(q.split())
print(query_words)
query_vocab = Counter(query_words)

response_words = []
# query_words = [q.split() for q in queries]
for r in responses:
  response_words.extend(r.split())
print(response_words)
response_vocab = Counter(response_words)

print(query_vocab)
print(response_vocab)

['oh my god', 'that is impossible', 'and what difference will it make', 'april 1912', "six o'clock", 'thank you daisy anna', 'just for once in my life', 'i would like to sleep until i woke up natural', ' is your fire still in  yes mrs patmore', 'ooh my will wonders never cease', " have you laid the servants' hall breakfast  yes mrs patmore", ' and finished blacking that stove  yes mrs patmore', ' what about the bedroom fires  all lit mrs patmore', 'right well take the things', ' where have you been  i am not late am i', 'daisy', ' breakfast is ready mr carson  ah william any papers yet', ' is the library tidy  yes mrs hughes', 'good', 'i want the dining room given a proper goingover today', 'you can do it when they have finished their breakfast', 'oh heavens girl', 'you are building a fire not inventing it', 'how many have you done', ' and they are off  no rest for the wicked', 'lady mary are the tea trays ready', "all ready mrs patmore if the water's boiled", 'could you give us a hand

In [ ]:
# queries = [p for p in queries if len(p) < MAX_SENTENCE_LENGTH]
# responses = [p for p in responses if len(p) < MAX_SENTENCE_LENGTH]


In [12]:
query_word_to_int = {}

word_int_label = 0
for word in query_vocab:
  if query_vocab[word] < THRESHOLD:
    continue

  else:
    query_word_to_int[word] = word_int_label
    word_int_label += 1

response_word_to_int = {}

word_int_label = 0
for word in response_vocab:
  if response_vocab[word] < THRESHOLD:
    continue

  else:
    response_word_to_int[word] = word_int_label
    word_int_label += 1
  

print(query_word_to_int)
print(response_word_to_int)
# EOS_token = 2  # End-of-sentence token

# class Voc:

{'oh': 0, 'my': 1, 'god': 2, 'that': 3, 'is': 4, 'and': 5, 'what': 6, 'will': 7, 'it': 8, 'make': 9, 'thank': 10, 'you': 11, 'daisy': 12, 'anna': 13, 'just': 14, 'for': 15, 'once': 16, 'in': 17, 'life': 18, 'i': 19, 'would': 20, 'like': 21, 'to': 22, 'until': 23, 'up': 24, 'your': 25, 'still': 26, 'yes': 27, 'mrs': 28, 'patmore': 29, 'never': 30, 'have': 31, 'the': 32, 'breakfast': 33, 'about': 34, 'all': 35, 'right': 36, 'well': 37, 'take': 38, 'things': 39, 'where': 40, 'been': 41, 'am': 42, 'not': 43, 'late': 44, 'ready': 45, 'mr': 46, 'carson': 47, 'ah': 48, 'william': 49, 'any': 50, 'papers': 51, 'hughes': 52, 'good': 53, 'want': 54, 'room': 55, 'given': 56, 'a': 57, 'can': 58, 'do': 59, 'when': 60, 'they': 61, 'their': 62, 'are': 63, 'how': 64, 'done': 65, 'off': 66, 'no': 67, 'lady': 68, 'mary': 69, 'if': 70, 'could': 71, 'give': 72, 'got': 73, 'her': 74, 'help': 75, 'at': 76, 'know': 77, 'but': 78, 'why': 79, 'cannot': 80, 'believe': 81, 'me': 82, 'neither': 83, 'his': 84, 'thi

In [13]:
spec_tokens = ['<PAD>','<SOS>','<EOS>','<UNK>']

for token in spec_tokens:
  query_word_to_int[token] = len(query_word_to_int)+1

for token in spec_tokens:
  response_word_to_int[token] = len(response_word_to_int)+1

In [14]:
# Inverse dictionary
query_int_to_word = {v:k for k,v in query_word_to_int.items()}
responses_int_to_word = {v:k for k,v in response_word_to_int.items()}

In [15]:
# pairs = list(zip(queries,responses))
pairs = []
for p in list(zip(queries,responses)):
  keep_query = True
  keep_response = True
  for x in p[0].split():
    if x not in query_word_to_int:
      keep_query = False
      break

  if not keep_query: continue
  for x in p[1].split():
    if x not in response_word_to_int:
      keep_response = False
      break

  if keep_query and keep_response:
    pairs.append(p)

print(pairs)


  

# for q in queries:
#   lst = [x for x in q if query_vocab[x] >= THRESHOLD])
#   if not lst: 
#     queries.remove(q)
#   else:
#     q = q + ' <EOS>'

# for r in responses:
#   r = ''.join([x for x in r if response_vocab[x] >= THRESHOLD])
#   if not r: 
#     responses.remove(r)
#   else:
#     r = r + ' <EOS>'

[('thank you daisy anna', 'just for once in my life'), (' where have you been  i am not late am i', 'you are late when i say you are late'), (' you are late  i know but', ' but what  you will see'), ('neither of them', 'you must tell mary'), ('but how can you manage', 'do not worry about that i can manage'), (' i can manage  all right mrs hughes', 'i will take over thank you'), ('of course you can', 'thomas take mr bates to his room and'), ('i would better go', 'tell her about james and patrick'), ('well i did not', 'no you did not'), ("there's nothing we can do about the title", 'no she cannot have the title'), ('no she cannot have the title', 'but she can have your money'), ('but she can have your money', 'and the estate'), ('i see', 'really edith'), ('well only if you do not tell', 'so has it all been settled'), ('they are not our family', 'well they are all the family i have got'), ('do you', 'i do not know'), ('well', 'i do not like to say'), ('i would but it will not be', 'thomas

In [ ]:
# pairs = list(zip(queries,responses))
# print(pairs)

[('oh my god', 'that is impossible'), ('that is impossible', 'i will take it up there now'), ('and what difference will it make', 'jimmy will do it when he comes in'), ('april 1912', "six o'clock"), ("six o'clock", 'thank you daisy anna'), ('thank you daisy anna', 'just for once in my life'), ('just for once in my life', 'i would like to sleep until i woke up natural'), ('i would like to sleep until i woke up natural', ' is your fire still in  yes mrs patmore'), (' is your fire still in  yes mrs patmore', 'ooh my will wonders never cease'), ('ooh my will wonders never cease', " have you laid the servants' hall breakfast  yes mrs patmore"), (" have you laid the servants' hall breakfast  yes mrs patmore", ' and finished blacking that stove  yes mrs patmore'), (' and finished blacking that stove  yes mrs patmore', ' what about the bedroom fires  all lit mrs patmore'), (' what about the bedroom fires  all lit mrs patmore', 'right well take the things'), ('right well take the things', 'and 

In [18]:
query_ints = []
for q in queries:
  temp = []
  for word in q:
    try:
      temp.append(query_word_to_int[word])
    except:
      temp.append(query_word_to_int['<UNK>'])
    query_ints.append(temp)

response_ints = []
for q in responses:
  temp = []
  for word in q:
    try:
      temp.append(query_word_to_int[word])
    except:
      temp.append(query_word_to_int['<UNK>'])
    response_ints.append(temp)

In [ ]:
pairs

[('oh my god', 'that is impossible'),
 ('that is impossible', 'i will take it up there now'),
 ('and what difference will it make', 'jimmy will do it when he comes in'),
 ('april 1912', "six o'clock"),
 ("six o'clock", 'thank you daisy anna'),
 ('thank you daisy anna', 'just for once in my life'),
 ('just for once in my life', 'i would like to sleep until i woke up natural'),
 ('i would like to sleep until i woke up natural',
  ' is your fire still in  yes mrs patmore'),
 (' is your fire still in  yes mrs patmore',
  'ooh my will wonders never cease'),
 ('ooh my will wonders never cease',
  " have you laid the servants' hall breakfast  yes mrs patmore"),
 (" have you laid the servants' hall breakfast  yes mrs patmore",
  ' and finished blacking that stove  yes mrs patmore'),
 (' and finished blacking that stove  yes mrs patmore',
  ' what about the bedroom fires  all lit mrs patmore'),
 (' what about the bedroom fires  all lit mrs patmore',
  'right well take the things'),
 ('right wel

In [ ]:
random.choice(pairs)

IndexError: ignored

In [16]:
def indexesFromSentence(voc, sentence):
    result = []
    for word in sentence.split():
        try:
          result.append(voc[word])
        except:
          # result.append(voc['<UNK>'])
          continue

    return result
    # return [voc[word] for word in sentence.split()]



def zeroPadding(l, voc,fillvalue='<PAD>'):
    return list(itertools.zip_longest(*l, fillvalue=voc[fillvalue]))

def binaryMatrix(l, value='<PAD>'):
    m = []
    for i, seq in enumerate(l):
        m.append([])
        for token in seq:
            if token == '<PAD>':
                m[i].append(0)
            else:
                m[i].append(1)
    return m

# Returns padded input sequence tensor and lengths
def inputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch,voc)
    padVar = torch.LongTensor(padList)
    return padVar, lengths

# Returns padded target sequence tensor, padding mask, and max target length
def outputVar(l, voc):
    indexes_batch = [indexesFromSentence(voc, sentence) for sentence in l]
    max_target_len = max([len(indexes) for indexes in indexes_batch])
    padList = zeroPadding(indexes_batch,voc)
    mask = binaryMatrix(padList)
    mask = torch.BoolTensor(mask)
    padVar = torch.LongTensor(padList)
    return padVar, mask, max_target_len

# Returns all items for a given batch of pairs
def batch2TrainData(voc, pair_batch):
    pair_batch.sort(key=lambda x: len(x[0].split(" ")), reverse=True)
    input_batch, output_batch = [], []
    for pair in pair_batch:
        input_batch.append(pair[0])
        output_batch.append(pair[1])
    inp, lengths = inputVar(input_batch, voc)
    output, mask, max_target_len = outputVar(output_batch, voc)
    return inp, lengths, output, mask, max_target_len


# Example for validation
small_batch_size = 5
query_word_to_int.update(response_word_to_int)
batches = batch2TrainData(query_word_to_int, [random.choice(pairs) for _ in range(small_batch_size)])
input_variable, lengths, target_variable, mask, max_target_len = batches

print("input_variable:", input_variable)
print("lengths:", lengths)
print("target_variable:", target_variable)
print("mask:", mask)
print("max_target_len:", max_target_len)

input_variable: tensor([[  2, 142, 142,  36,   9],
        [  3,  26,  26,  43,  14],
        [  9, 223, 223,  14, 233],
        [  0, 142, 142,  47, 233],
        [150, 233, 233,  16, 233],
        [150, 233, 233, 233, 233],
        [ 13, 233, 233, 233, 233],
        [ 14, 233, 233, 233, 233],
        [ 20, 233, 233, 233, 233],
        [173, 233, 233, 233, 233]])
lengths: tensor([10,  4,  4,  5,  2])
target_variable: tensor([[  2, 142, 142,   2,   2],
        [ 56,  50,  50,  87,   9],
        [  9, 233, 233,   2,  59],
        [  5, 233, 233,  22,  74],
        [  2, 233, 233,   4, 233],
        [ 92, 233, 233, 155, 233],
        [ 83, 233, 233,   6, 233],
        [233, 233, 233,  24, 233],
        [233, 233, 233, 121, 233],
        [233, 233, 233, 129, 233]])
mask: tensor([[True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, True, True, True],
        [True, True, 

In [30]:
class EncoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding, n_layers=1, dropout=0):
        super(EncoderRNN, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.embedding = embedding

        # Initialize GRU; the input_size and hidden_size params are both set to 'hidden_size'
        #   because our input size is a word embedding with number of features == hidden_size
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers,
                          dropout=(0 if n_layers == 1 else dropout), bidirectional=True)

    def forward(self, input_seq, input_lengths, hidden=None):
        # Convert word indexes to embeddings
        embedded = self.embedding(input_seq)
        # Pack padded batch of sequences for RNN module
        packed = nn.utils.rnn.pack_padded_sequence(embedded, input_lengths,enforce_sorted=False)
        # Forward pass through GRU
        outputs, hidden = self.gru(packed, hidden)
        # Unpack padding
        outputs, _ = nn.utils.rnn.pad_packed_sequence(outputs)
        # Sum bidirectional GRU outputs
        outputs = outputs[:, :, :self.hidden_size] + outputs[:, : ,self.hidden_size:]
        # Return output and final hidden state
        return outputs, hidden

In [31]:
# Luong attention layer
class Attn(nn.Module):
    def __init__(self, method, hidden_size):
        super(Attn, self).__init__()
        self.method = method
        if self.method not in ['dot', 'general', 'concat']:
            raise ValueError(self.method, "is not an appropriate attention method.")
        self.hidden_size = hidden_size
        if self.method == 'general':
            self.attn = nn.Linear(self.hidden_size, hidden_size)
        elif self.method == 'concat':
            self.attn = nn.Linear(self.hidden_size * 2, hidden_size)
            self.v = nn.Parameter(torch.FloatTensor(hidden_size))

    def dot_score(self, hidden, encoder_output):
        return torch.sum(hidden * encoder_output, dim=2)

    def general_score(self, hidden, encoder_output):
        energy = self.attn(encoder_output)
        return torch.sum(hidden * energy, dim=2)

    def concat_score(self, hidden, encoder_output):
        energy = self.attn(torch.cat((hidden.expand(encoder_output.size(0), -1, -1), encoder_output), 2)).tanh()
        return torch.sum(self.v * energy, dim=2)

    def forward(self, hidden, encoder_outputs):
        # Calculate the attention weights (energies) based on the given method
        if self.method == 'general':
            attn_energies = self.general_score(hidden, encoder_outputs)
        elif self.method == 'concat':
            attn_energies = self.concat_score(hidden, encoder_outputs)
        elif self.method == 'dot':
            attn_energies = self.dot_score(hidden, encoder_outputs)

        # Transpose max_length and batch_size dimensions
        attn_energies = attn_energies.t()

        # Return the softmax normalized probability scores (with added dimension)
        return F.softmax(attn_energies, dim=1).unsqueeze(1)

In [32]:
class LuongAttnDecoderRNN(nn.Module):
    def __init__(self, attn_model, embedding, hidden_size, output_size, n_layers=1, dropout=0.1):
        super(LuongAttnDecoderRNN, self).__init__()

        # Keep for reference
        self.attn_model = attn_model
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.n_layers = n_layers
        self.dropout = dropout

        # Define layers
        self.embedding = embedding
        self.embedding_dropout = nn.Dropout(dropout)
        self.gru = nn.GRU(hidden_size, hidden_size, n_layers, dropout=(0 if n_layers == 1 else dropout))
        self.concat = nn.Linear(hidden_size * 2, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)

        self.attn = Attn(attn_model, hidden_size)

    def forward(self, input_step, last_hidden, encoder_outputs):
        # Note: we run this one step (word) at a time
        # Get embedding of current input word
        embedded = self.embedding(input_step)
        embedded = self.embedding_dropout(embedded)
        # Forward through unidirectional GRU
        rnn_output, hidden = self.gru(embedded, last_hidden)
        # Calculate attention weights from the current GRU output
        attn_weights = self.attn(rnn_output, encoder_outputs)
        # Multiply attention weights to encoder outputs to get new "weighted sum" context vector
        context = attn_weights.bmm(encoder_outputs.transpose(0, 1))
        # Concatenate weighted context vector and GRU output using Luong eq. 5
        rnn_output = rnn_output.squeeze(0)
        context = context.squeeze(1)
        concat_input = torch.cat((rnn_output, context), 1)
        concat_output = torch.tanh(self.concat(concat_input))
        # Predict next word using Luong eq. 6
        output = self.out(concat_output)
        output = F.softmax(output, dim=1)
        # Return output and final hidden state
        return output, hidden

In [33]:
def maskNLLLoss(inp, target, mask):
    nTotal = mask.sum()
    crossEntropy = -torch.log(torch.gather(inp, 1, target.view(-1, 1)).squeeze(1))
    loss = crossEntropy.masked_select(mask).mean()
    loss = loss.to(device)
    return loss, nTotal.item()

In [41]:
def train(input_variable, lengths, target_variable, mask, max_target_len, encoder, decoder, embedding,
          encoder_optimizer, decoder_optimizer, batch_size, clip, max_length=MAX_LENGTH):

    # Zero gradients
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    # Set device options
    input_variable = input_variable.to(device)
    lengths = lengths.to(device)
    target_variable = target_variable.to(device)
    mask = mask.to(device)

    # Initialize variables
    loss = 0
    print_losses = []
    n_totals = 0

    # Forward pass through encoder
    encoder_outputs, encoder_hidden = encoder(input_variable, lengths)

    # Create initial decoder input (start with SOS tokens for each sentence)
    decoder_input = torch.LongTensor([[query_word_to_int['<SOS>'] for _ in range(batch_size)]])
    decoder_input = decoder_input.to(device)

    # Set initial decoder hidden state to the encoder's final hidden state
    decoder_hidden = encoder_hidden[:decoder.n_layers]

    # Determine if we are using teacher forcing this iteration
    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    # Forward batch of sequences through decoder one time step at a time
    if use_teacher_forcing:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # Teacher forcing: next input is current target
            decoder_input = target_variable[t].view(1, -1)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal
    else:
        for t in range(max_target_len):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden, encoder_outputs
            )
            # No teacher forcing: next input is decoder's own current output
            _, topi = decoder_output.topk(1)
            decoder_input = torch.LongTensor([[topi[i][0] for i in range(batch_size)]])
            decoder_input = decoder_input.to(device)
            # Calculate and accumulate loss
            mask_loss, nTotal = maskNLLLoss(decoder_output, target_variable[t], mask[t])
            loss += mask_loss
            print_losses.append(mask_loss.item() * nTotal)
            n_totals += nTotal

    # Perform backpropatation
    loss.backward()

    # Clip gradients: gradients are modified in place
    _ = nn.utils.clip_grad_norm_(encoder.parameters(), clip)
    _ = nn.utils.clip_grad_norm_(decoder.parameters(), clip)

    # Adjust model weights
    encoder_optimizer.step()
    decoder_optimizer.step()

    return sum(print_losses) / n_totals

In [48]:
def trainIters(model_name, voc, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer, embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size, print_every, save_every, clip, corpus_name, loadFilename):

    # Load batches for each iteration
    training_batches = [batch2TrainData(voc, [random.choice(pairs) for _ in range(batch_size)])
                      for _ in range(n_iteration)]

    # Initializations
    print('Initializing ...')
    start_iteration = 1
    print_loss = 0
    if loadFilename:
        start_iteration = checkpoint['iteration'] + 1

    # Training loop
    print("Training...")
    for iteration in range(start_iteration, n_iteration + 1):
        training_batch = training_batches[iteration - 1]
        # Extract fields from batch
        input_variable, lengths, target_variable, mask, max_target_len = training_batch

        # Run a training iteration with batch
        loss = train(input_variable, lengths, target_variable, mask, max_target_len, encoder,
                     decoder, embedding, encoder_optimizer, decoder_optimizer, batch_size, clip)
        print_loss += loss

        # Print progress
        if iteration % print_every == 0:
            print_loss_avg = print_loss / print_every
            print("Iteration: {}; Percent complete: {:.1f}%; Average loss: {:.4f}".format(iteration, iteration / n_iteration * 100, print_loss_avg))
            print_loss = 0

        # Save checkpoint
        if (iteration % save_every == 0):
            directory = os.path.join(save_dir, model_name, corpus_name, '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size))
            if not os.path.exists(directory):
                os.makedirs(directory)
            torch.save({
                'iteration': iteration,
                'en': encoder.state_dict(),
                'de': decoder.state_dict(),
                'en_opt': encoder_optimizer.state_dict(),
                'de_opt': decoder_optimizer.state_dict(),
                'loss': loss,
                'voc_dict': voc,
                'embedding': embedding.state_dict()
            }, os.path.join(directory, '{}_{}.tar'.format(iteration, 'checkpoint')))

In [43]:
class GreedySearchDecoder(nn.Module):
    def __init__(self, encoder, decoder):
        super(GreedySearchDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder

    def forward(self, input_seq, input_length, max_length):
        # Forward input through encoder model
        encoder_outputs, encoder_hidden = self.encoder(input_seq, input_length)
        # Prepare encoder's final hidden layer to be first hidden input to the decoder
        decoder_hidden = encoder_hidden[:decoder.n_layers]
        # Initialize decoder input with SOS_token
        decoder_input = torch.ones(1, 1, device=device, dtype=torch.long) * SOS_token
        # Initialize tensors to append decoded words to
        all_tokens = torch.zeros([0], device=device, dtype=torch.long)
        all_scores = torch.zeros([0], device=device)
        # Iteratively decode one word token at a time
        for _ in range(max_length):
            # Forward pass through decoder
            decoder_output, decoder_hidden = self.decoder(decoder_input, decoder_hidden, encoder_outputs)
            # Obtain most likely word token and its softmax score
            decoder_scores, decoder_input = torch.max(decoder_output, dim=1)
            # Record token and score
            all_tokens = torch.cat((all_tokens, decoder_input), dim=0)
            all_scores = torch.cat((all_scores, decoder_scores), dim=0)
            # Prepare current token to be next decoder input (add a dimension)
            decoder_input = torch.unsqueeze(decoder_input, 0)
        # Return collections of word tokens and scores
        return all_tokens, all_scores

In [53]:
def evaluate(encoder, decoder, searcher, voc, sentence, max_length=MAX_LENGTH):
    ### Format input sentence as a batch
    # words -> indexes
    indexes_batch = [indexesFromSentence(voc, sentence)]
    # Create lengths tensor
    lengths = torch.tensor([len(indexes) for indexes in indexes_batch])
    # Transpose dimensions of batch to match models' expectations
    input_batch = torch.LongTensor(indexes_batch).transpose(0, 1)
    # Use appropriate device
    input_batch = input_batch.to(device)
    lengths = lengths.to(device)
    # Decode sentence with searcher
    tokens, scores = searcher(input_batch, lengths, max_length)
    # indexes -> words
    decoded_words = [voc.index2word[token.item()] for token in tokens]
    return decoded_words


def evaluateInput(encoder, decoder, searcher, voc):
    input_sentence = ''
    while(1):
        try:
            # Get input sentence
            input_sentence = input('> ')
            # Check if it is quit case
            if input_sentence == 'q' or input_sentence == 'quit': break
            # Normalize sentence
            input_sentence = clean_text(input_sentence)
            # Evaluate sentence
            output_words = evaluate(encoder, decoder, searcher, voc, input_sentence)
            # Format and print response sentence
            output_words[:] = [x for x in output_words if not (x == 'EOS' or x == 'PAD')]
            print('Bot:', ' '.join(output_words))

        except KeyError:
            print("Error: Encountered unknown word.")

In [45]:
# Configure models
model_name = 'cb_model'
attn_model = 'dot'
#attn_model = 'general'
#attn_model = 'concat'
hidden_size = 500
encoder_n_layers = 2
decoder_n_layers = 2
dropout = 0.1
batch_size = 64

# Set checkpoint to load from; set to None if starting from scratch
loadFilename = None
checkpoint_iter = 4000
#loadFilename = os.path.join(save_dir, model_name, corpus_name,
#                            '{}-{}_{}'.format(encoder_n_layers, decoder_n_layers, hidden_size),
#                            '{}_checkpoint.tar'.format(checkpoint_iter))


# Load model if a loadFilename is provided
if loadFilename:
    # If loading on same machine the model was trained on
    checkpoint = torch.load(loadFilename)
    # If loading a model trained on GPU to CPU
    #checkpoint = torch.load(loadFilename, map_location=torch.device('cpu'))
    encoder_sd = checkpoint['en']
    decoder_sd = checkpoint['de']
    encoder_optimizer_sd = checkpoint['en_opt']
    decoder_optimizer_sd = checkpoint['de_opt']
    embedding_sd = checkpoint['embedding']
    voc.__dict__ = checkpoint['voc_dict']


print('Building encoder and decoder ...')
# Initialize word embeddings
embedding = nn.Embedding(len(query_word_to_int.keys()), hidden_size)
if loadFilename:
    embedding.load_state_dict(embedding_sd)
# Initialize encoder & decoder models
encoder = EncoderRNN(hidden_size, embedding, encoder_n_layers, dropout)
decoder = LuongAttnDecoderRNN(attn_model, embedding, hidden_size, len(query_word_to_int.keys()), decoder_n_layers, dropout)
if loadFilename:
    encoder.load_state_dict(encoder_sd)
    decoder.load_state_dict(decoder_sd)
# Use appropriate device
encoder = encoder.to(device)
decoder = decoder.to(device)
print('Models built and ready to go!')

Building encoder and decoder ...
Models built and ready to go!


In [46]:
save_dir = os.path.join("data", "save")
corpus_name = "downton abbey"

In [49]:
# Configure training/optimization
clip = 50.0
teacher_forcing_ratio = 1.0
learning_rate = 0.0001
decoder_learning_ratio = 5.0
n_iteration = 4000
print_every = 1
save_every = 500

# Ensure dropout layers are in train mode
encoder.train()
decoder.train()

# Initialize optimizers
print('Building optimizers ...')
encoder_optimizer = optim.Adam(encoder.parameters(), lr=learning_rate)
decoder_optimizer = optim.Adam(decoder.parameters(), lr=learning_rate * decoder_learning_ratio)
if loadFilename:
    encoder_optimizer.load_state_dict(encoder_optimizer_sd)
    decoder_optimizer.load_state_dict(decoder_optimizer_sd)

# If you have cuda, configure cuda to call
for state in encoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()

for state in decoder_optimizer.state.values():
    for k, v in state.items():
        if isinstance(v, torch.Tensor):
            state[k] = v.cuda()
    
# Run training iterations
print("Starting Training!")
trainIters(model_name, query_word_to_int, pairs, encoder, decoder, encoder_optimizer, decoder_optimizer,
           embedding, encoder_n_layers, decoder_n_layers, save_dir, n_iteration, batch_size,
           print_every, save_every, clip, corpus_name, loadFilename)

Building optimizers ...
Starting Training!
Initializing ...
Training...
Iteration: 1; Percent complete: 0.0%; Average loss: 0.0022
Iteration: 2; Percent complete: 0.1%; Average loss: 0.0073
Iteration: 3; Percent complete: 0.1%; Average loss: 0.0018
Iteration: 4; Percent complete: 0.1%; Average loss: 0.0172
Iteration: 5; Percent complete: 0.1%; Average loss: 0.0023
Iteration: 6; Percent complete: 0.1%; Average loss: 0.0020
Iteration: 7; Percent complete: 0.2%; Average loss: 0.0034
Iteration: 8; Percent complete: 0.2%; Average loss: 0.0032
Iteration: 9; Percent complete: 0.2%; Average loss: 0.0023
Iteration: 10; Percent complete: 0.2%; Average loss: 0.0024
Iteration: 11; Percent complete: 0.3%; Average loss: 0.0023
Iteration: 12; Percent complete: 0.3%; Average loss: 0.0015
Iteration: 13; Percent complete: 0.3%; Average loss: 0.0014
Iteration: 14; Percent complete: 0.4%; Average loss: 0.0014
Iteration: 15; Percent complete: 0.4%; Average loss: 0.0015
Iteration: 16; Percent complete: 0.4%

KeyboardInterrupt: ignored

In [54]:
# Set dropout layers to eval mode
encoder.eval()
decoder.eval()

# Initialize search module
searcher = GreedySearchDecoder(encoder, decoder)

# Begin chatting (uncomment and run the following line to begin)
voc = query_word_to_int
evaluateInput(encoder, decoder, searcher, voc)

> hi


RuntimeError: ignored